In [7]:
# Simple propositional Wumpus World Reasoning
# Query1: Is there a Pit at (1,2)?
# Query2: Is there a Wumpus at (2,2)?

# Each cell has knowledge: S=Stench, B=Breeze, P=PIT, W=WUMPUS, V=Visited, G=Glitter
# We use '~' to mark negations, '?' for uncertainty

import copy

# Initialize the 4x4 grid
def create_world():
    grid = [[{'S': None, 'B': None, 'P': None, 'W': None, 'V': False, 'G': None}
             for _ in range(4)] for _ in range(4)]
    return grid

def print_world(grid, agent_pos):
    print(f"\nAgent is on: {agent_pos[0]+1}, {agent_pos[1]+1}")
    for y in range(3, -1, -1):  # print top row first
        row = []
        for x in range(4):
            cell = grid[y][x]
            if cell['V']:
                symbols = []
                for key in ['S','B','P','W','V','G']:
                    val = cell[key]
                    if val is True:
                        symbols.append(key)
                    elif val is False:
                        symbols.append("~"+key)
                    elif val == "?":
                        symbols.append(key+"?")
                row.append(",".join(symbols))
            else:
                row.append("   ")
        print(" | ".join(f"{r:<18}" for r in row))
    print("-"*80)

def neighbors(x, y):
    dirs = [(0,1),(1,0),(-1,0),(0,-1)]
    return [(x+dx, y+dy) for dx,dy in dirs if 0 <= x+dx < 4 and 0 <= y+dy < 4]

def infer_rules(grid, x, y):
    cell = grid[y][x]
    # Breeze means adjacent pits
    if cell['B'] is True:
        for (nx, ny) in neighbors(x, y):
            if grid[ny][nx]['P'] is None:
                grid[ny][nx]['P'] = "?"
    elif cell['B'] is False:
        for (nx, ny) in neighbors(x, y):
            grid[ny][nx]['P'] = False
    # Stench means adjacent Wumpus
    if cell['S'] is True:
        for (nx, ny) in neighbors(x, y):
            if grid[ny][nx]['W'] is None:
                grid[ny][nx]['W'] = "?"
    elif cell['S'] is False:
        for (nx, ny) in neighbors(x, y):
            grid[ny][nx]['W'] = False

def wumpus_demo():
    world = create_world()
    path = [(0,0),(1,0),(0,1),(1,1),(2,1)]
    perceptions = {
        (0,0): {'S': False, 'B': False},
        (1,0): {'S': True,  'B': False},
        (0,1): {'S': False, 'B': True},
        (1,1): {'S': True,  'B': True},
        (2,1): {'S': False, 'B': False},
    }

    for (x,y) in path:
        world[y][x]['V'] = True
        for k,v in perceptions[(x,y)].items():
            world[y][x][k] = v
        infer_rules(world, x, y)
        print_world(world, (x,y))

    # Queries
    pit_12 = world[1][0]['P']  # (1,2)
    wumpus_22 = world[1][1]['W']  # (2,2)
    print("\nFinal Entailment:")
    print(f"Query1: Pit at (1,2)?  => {pit_12}")
    print(f"Query2: Wumpus at (2,2)?  => {wumpus_22}")

wumpus_demo()



Agent is on: 1, 1
                   |                    |                    |                   
                   |                    |                    |                   
                   |                    |                    |                   
~S,~B,V            |                    |                    |                   
--------------------------------------------------------------------------------

Agent is on: 2, 1
                   |                    |                    |                   
                   |                    |                    |                   
                   |                    |                    |                   
~S,~B,~P,W?,V      | S,~B,~P,~W,V       |                    |                   
--------------------------------------------------------------------------------

Agent is on: 1, 2
                   |                    |                    |                   
                   |                    |  

In [4]:
import itertools

# -----------------------------------------------------------
# Step 1: Define propositional variables
# -----------------------------------------------------------
symbols = [
    "P11", "P12", "P21", "P22", "W11", "W12", "W21", "W22",
    "B11", "B21", "B12", "S11", "S21", "S12"
]

# Helper for combinations
def all_models(symbols):
    for vals in itertools.product([True, False], repeat=len(symbols)):
        yield dict(zip(symbols, vals))

# -----------------------------------------------------------
# Step 2: Define the knowledge base as a logical function
# -----------------------------------------------------------
def KB(model):
    # Define shorthand for readability
    P11, P12, P21, P22 = model["P11"], model["P12"], model["P21"], model["P22"]
    W11, W12, W21, W22 = model["W11"], model["W12"], model["W21"], model["W22"]
    B11, B21, B12 = model["B11"], model["B21"], model["B12"]
    S11, S21, S12 = model["S11"], model["S21"], model["S12"]

    # Knowledge rules
    rule1 = (B11 == (P12 or P21))
    rule2 = (B21 == (P11 or P22))
    rule3 = (B12 == (P11 or P22))
    rule4 = (S11 == (W12 or W21))
    rule5 = (S21 == (W11 or W22))
    rule6 = (S12 == (W11 or W22))

    # Percepts: known facts from the agent
    facts = (
        (not B11) and  # No breeze at (1,1)
        (not S11) and  # No stench at (1,1)
        B21 and        # Breeze at (2,1)
        S12            # Stench at (1,2)
    )

    return rule1 and rule2 and rule3 and rule4 and rule5 and rule6 and facts

# -----------------------------------------------------------
# Step 3: Model checking for entailment
# -----------------------------------------------------------
def entails(KB, query):
    valid_models = []
    for model in all_models(symbols):
        if KB(model):  # model satisfies the KB
            valid_models.append(model)
            if not query(model):  # KB true but query false → not entailed
                return False, valid_models
    return True, valid_models

# -----------------------------------------------------------
# Step 4: Define queries
# -----------------------------------------------------------
def query_pit_12(model):
    return model["P12"]

def query_wumpus_22(model):
    return model["W22"]

# -----------------------------------------------------------
# Step 5: Run checks
# -----------------------------------------------------------
entails_pit, models_pit = entails(KB, query_pit_12)
entails_wumpus, models_wumpus = entails(KB, query_wumpus_22)

# -----------------------------------------------------------
# Step 6: Display results
# -----------------------------------------------------------
print("🧩 Knowledge Base Models that Satisfy Given Facts:")
print(f"Found {len(models_pit)} possible models.")

print("\nQuery 1: Is there a Pit at (1,2)?")
print("Entailed by KB?" , entails_pit)

print("\nQuery 2: Is there a Wumpus at (2,2)?")
print("Entailed by KB?" , entails_wumpus)


🧩 Knowledge Base Models that Satisfy Given Facts:
Found 1 possible models.

Query 1: Is there a Pit at (1,2)?
Entailed by KB? False

Query 2: Is there a Wumpus at (2,2)?
Entailed by KB? False
